In [ ]:
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import networkx as nx
import EoN

#plt.style.use("plot_style.txt")

In [ ]:
"""
1) Take the square root of the number X; we'll call it N.
2) Set N equal to the ceiling of N (round up to the nearest integer).
3) Test for (X % N). If N divides evenly into X, we found our first number.
  if 0, divide X by N to get M. M and N are our numbers
  if not 0, increment N by 1 and start step 3 over.
https://stackoverflow.com/questions/16266931/input-an-integer-find-the-two-closest-integers-which-when-multiplied-equal-th
"""
def sqrt_int(X: int):
    N = np.floor(np.sqrt(X))
    while bool(X % N):
        N -= 1
    M = X // N
    return M, N

In [ ]:
def run_simulation1(x=100.0, y=100.0, initial_infected=0.05, infection_rate=2.0, recovery_rate=1.0, vaccination_rate=0.02):
    G = nx.grid_2d_graph(x, y)

    #the spontaneous transitions
    H = nx.DiGraph()
    H.add_edge('S', 'V', rate=vaccination_rate)
    H.add_edge('I', 'R', rate=recovery_rate)  # gamma  

    #the induced transitions
    J = nx.DiGraph()
    J.add_edge(('I', 'S'), ('I', 'I'), rate=infection_rate) # tau

    bounds = np.ceil(np.divide(np.array(sqrt_int(x * y * initial_infected)), 2))
    x_lower_bound = np.floor(np.divide(x, 2)) - bounds[0]
    x_upper_bound = np.floor(np.divide(x, 2)) + bounds[0]
    y_lower_bound = np.floor(np.divide(y, 2)) - bounds[1]
    y_upper_bound = np.floor(np.divide(y, 2)) + bounds[1]

    #we'll initially infect those near the middle
    IC = defaultdict(lambda: 'S')
    initial_infections = [(u, v) for (u, v) in G if x_lower_bound < u < x_upper_bound and y_lower_bound < v < y_upper_bound]
    for node in initial_infections:
        IC[node] = 'I'

    initial_infections_count = len(initial_infections)
    initial_infections_percentage = np.divide(len(initial_infections), x*y)
    print("Initial infected population: {} ~ {:.2%}".format(initial_infections_count, initial_infections_percentage))

    
    #create simulation
    pos = {node:node for node in G}
    return_statuses = ['S', 'I', 'R', 'V']
    color_dict = {'S': '#009a80','I':'#ff2000', 'R':'gray','V': '#5AB3E6'}
    sim_kwargs = {'color_dict': color_dict, 'pos': pos, 'tex': False}

    simulation = EoN.Gillespie_simple_contagion(G, H, J, IC, return_statuses, tmax=100, return_full_data=True, sim_kwargs=sim_kwargs)

    return (initial_infections_count, initial_infections_percentage, simulation)

In [ ]:
def plot_simulation_metrics(sim, filename):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 5))

    t = sim.t()
    S = sim._summary_[1]['S']
    I = sim._summary_[1]['I']
    R = sim._summary_[1]['R']
    V = sim._summary_[1]['V']

    # SIRV
    ax1.plot(t, S, label="Susceptible")
    ax1.plot(t, I, label="Infectious")
    ax1.plot(t, R, label="Recovered")
    ax1.plot(t, V, label="Vaccinated")

    ax1.set_title("SIRV")
    ax1.set_xlim([t[0], t[-1]])
    ax1.set_ylim([0, S[0]])
    ax1.legend()

    # Infectious
    xpos = t[np.argmax(I)]

    ax2.plot(t, I)
    ax2.axvline(x=xpos, c="black")
    ax2.text(x=xpos + 1, y=25, s=str(np.max(I)))

    ax2.set_title("Infectious (I) Compartment")
    ax2.set_xlim([t[0], t[-1]])
    ax2.set_ylim([0, np.max(I)])

    plt.tight_layout()
    plt.savefig(filename)
    plt.show()

In [ ]:
def plot_simulation_graph(sim, time, filename, node_size=4):
    sim.display(time, node_size=node_size, ts_plots=[['S', 'I', 'R', 'V']])
    plt.savefig(filename)

In [ ]:
def render_simulation(sim, filename, node_size=4):
    ani = sim.animate(frame_times=np.linspace(0, sim._t_[-1], 201), ts_plots=[['S', 'I', 'R', 'V']], node_size=node_size)
    ani.save(filename, fps=60, dpi=200, bitrate=1000, extra_args=['-vcodec', 'libx264', '-pix_fmt', 'yuv420p'])

In [ ]:
grids = [(x * 1000, *sqrt_int(x * 1000)) for x in [7, 11, 30, 65, 85, 120]]
pd.DataFrame(grids, columns=["N", "x", "y"])

In [ ]:
# random test
#grid_x, grid_y = 64, 125
#vac = 0.01
#beta = 2.0
#gamma = 1.0

# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7570398/
#grid_x, grid_y = 64, 125
#beta = 0.0561215
#gamma = 0.0455331
#vac = 0.1 # random

grid_x, grid_y = 100, 70
vac = 0.0041
filename = "sirv-sim1-s1-p7"

# Phase 41, 23Jun2021-01Jul2021 (most infectious) [SKENARIO 1]
beta = 0.156533
gamma = 0.018125

# Phase 47, 24Aug2021-07Sep2021 (latest) [SKENARIO 2]
#beta = 0.037300
#gamma = 0.092772

sim1 = run_simulation1(grid_x, grid_y, 0.05, beta, gamma, vac)

In [ ]:
plot_simulation_metrics(sim1, "img-2/" + filename + "-graph.jpg")

In [ ]:
plot_simulation_graph(sim1, 100, "img-2/" + filename + "-spread.jpg")

In [ ]:
render_simulation(sim1, "render/" + filename + ".mp4")

In [ ]:
#run all scenarios
vac = 0.0041
grid_run = [
    (100, 70, 0.156533, 0.018125, "sirv-sim1-s1-p7"),
    (100, 70, 0.037300, 0.092772, "sirv-sim1-s2-p7"),
    (110, 100, 0.156533, 0.018125, "sirv-sim2-s1-p11"),
    (110, 100, 0.037300, 0.092772, "sirv-sim2-s2-p11"),
    (200, 150, 0.156533, 0.018125, "sirv-sim3-s1-p30"),
    (200, 150, 0.037300, 0.092772, "sirv-sim3-s2-p30"),

    (260, 250, 0.037300, 0.092772, "sirv-sim4-s2-p65"),
    (340, 250, 0.037300, 0.092772, "sirv-sim5-s2-p85"),
    (375, 320, 0.037300, 0.092772, "sirv-sim6-s2-p120"),
]

results = []

for grid_x, grid_y, beta, gamma, simname in grid_run:
    inf_initial_count, inf_initial_percentage, sim = run_simulation1(grid_x, grid_y, 0.05, beta, gamma, vac)
    total_inf = (sim.I() + sim.R())[-1]
    top_inf = np.max(sim.I())

    print("Simulation:       ", simname)
    print("Infection rate:   ", beta)
    print("Recovery rate:    ", gamma)
    print("Vaccination rate: ", vac)
    print("Initial infected: ", inf_initial_count, " ~ ", inf_initial_percentage)
    print("Most infected:    ", top_inf)
    print("Total infected:   ", total_inf)

    results.append((simname, beta, gamma, vac, grid_x * grid_y, inf_initial_count, inf_initial_percentage, top_inf, total_inf))

    plot_simulation_metrics(sim, "img-2/" + simname + "-graph.jpg")
    plot_simulation_graph(sim, 100, "img-2/" + simname + "-spread.jpg")
    render_simulation(sim, "render/" + simname + ".mp4")

In [ ]:
df = pd.DataFrame(results, columns=["simulation", "beta", "gamma", "vac", "population", "inf_initial_count", "inf_initial_percentage", "top_inf", "total_inf"])
df["inf_ratio"] = df["total_inf"] / df["population"]
df.to_csv("sim_summary.csv", index=None)
df